# 心理咨询论坛问答语料 - 制作迁徙学习数据集

为 NVIDIA/Megatron-LM 准备该领域的 finetune 数据集

## 要点

- 使用教少量的心理咨询论坛问答数据，在之前的泛情绪类预训练模型基础上进行 finetune
- 使用特殊标记(Token) 分隔问题标题、文本、答案文本

## Importings

In [1]:
import os
import json
import random
from copy import copy
from contextlib import closing, ExitStack
from functools import partial
from glob import glob, iglob
from itertools import chain, repeat
from multiprocessing import Pool

import pandas as pd
import sentencepiece as spm
from tqdm.auto import tqdm

from IPython.display import display

## Tokenizer

使用与 Pretrained-Model 同样的 tokenizer

In [2]:
spm_model_path = '../data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'

tokenizer = spm.SentencePieceProcessor()
tokenizer.load(spm_model_path)

True

### 特殊标记

该 Tokenizer 既有的特殊标记有：

In [3]:
for i in range(8):
  print(i, tokenizer.id_to_piece(i), tokenizer.is_control(i))

0 <pad> True
1 <unk> False
2 <bos> True
3 <eos> True
4 <sep> False
5 <cls> False
6 <|endoftext|> False
7 一个 False


我们选用：

- 使用`id=4 <sep>` 表示问题部分中，标题与正文的分隔
- 使用`id=6 <|endoftext|>` 表示问题结束
- 使用 `id=2 <bos>` 表示回答的开始

In [4]:
SEP = tokenizer.id_to_piece(4)
EOT = tokenizer.id_to_piece(6)
BOS = tokenizer.id_to_piece(2)

## Corpus

In [5]:
corpus_files = glob('/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/*.json')

In [6]:
print('语料文件:')
display(corpus_files)
print()

it = chain.from_iterable(map(open, corpus_files))
total = sum(1 for _ in tqdm(it, unit='line'))

print(f'总行数: {total:,d}')

语料文件:


['/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/valid_xinli_qax.json',
 '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/train_xinli_qax.json',
 '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/test_xinli_qax.json']


总行数: 187,156


现在预览语料数据格式，随机选择一个：

In [7]:
n = random.randint(0, total)

it = chain.from_iterable(map(open, corpus_files))
it = enumerate(it)
for i, line in tqdm(it, total=n):
    if i < n:
        continue
    data = json.loads(line)
    display(data)
    break


{'topics': ['治疗', '疾病诊断', '心理危机'],
 'title': ['抑郁药吃过量的会死吗'],
 'text': ['有一个人吃了三十多片多虑平 他会死吗 还是会昏睡几天 用不用医院 洗胃后会不会留下后遗症'],
 'answers': [['你想试试，', '可以先找好医院再试，', '如果愿意可以选择咨询。']]}

## 样本选用和格式转换

我们应：

- 选取问题+回答构成输入样本
- 一个问题可以对应多个回答
- 不要太短的
- 不要太长的

在此基础上，选用 `10,000` 个样本用于今次的开发

In [8]:
### 长短限制定义：

MIN_QUESTION_LENGTH = 16
MAX_QUESTION_LENGTH = 512
MIN_ANSWER_LENGTH = 32
MAX_LENGTH = 1024+128


def extract_samples(line):
    line = line.strip()
    if not line:
        return []
    data = json.loads(line)
    result = []
    # 没有回答的不要
    if not data.get('answers'):
        return []
    # 问题:
    title = ''.join(data['title']).strip()
    # 没有标题的不要
    if not title:
        return []
    text = ''.join(data['text']).strip()
    # 没有问题内容的不要
    if not text:
        return []
    n_title = len(tokenizer.encode_as_ids(title))
    n_text = len(tokenizer.encode_as_ids(text))
    # 太短不要
    if n_title + n_text < MIN_QUESTION_LENGTH:
        return []
    # 太长不要
    if n_title + n_text > MAX_QUESTION_LENGTH:
        return []
    #
    question_string = title + SEP + text + EOT
    # 答案
    for answer in data.get('answers', []):
        text = ''.join(answer).strip()
        length = len(tokenizer.encode_as_ids(text))
        # 太短不要
        if length < MIN_ANSWER_LENGTH:
            continue
        # 太长不要
        if n_title + n_text + length > MAX_LENGTH:
            continue
        result.append(question_string + BOS + text)
    #
    return result

用上一次随机取得的行试试看：

In [9]:
extract_samples(line)

[]

## 计算全部样本

In [10]:
def generate_samples_from_corpus():
    lines_iter = chain.from_iterable(map(open, corpus_files))
    lines_iter = tqdm(lines_iter, desc='MAP', total=total)
    with Pool() as pool:
        mapping_iter = pool.imap_unordered(extract_samples, lines_iter)
        mapping_iter = tqdm(mapping_iter, desc='RDC', total=total)
        yield from chain.from_iterable(mapping_iter)

samples = []
for sample in generate_samples_from_corpus():
    samples.append(sample)

print(f'满足要求的样本数: {len(samples):,d}')



满足要求的样本数: 220,840


随机显示一个:

In [11]:
n = random.randint(0, len(samples))
s = samples[n]
print(s)

毕业找工作和考国网<sep>我大学学的专业是绝缘与电缆，这个专业就业面窄，也就是去电缆厂做技术员，实习一次后，觉得不适应这种工作，打算考国网，因为学校不是重点，考上了基本也就是去县局，现在很迷茫，不知道该往哪个方向走～<|endoftext|><bos>能找到工作，能养活自己就是好样的要想挑肥拣瘦的话，要么退回高考，考个好学校再选个好专业，要么靠自己毕业后去创业去奋斗


## 采样并存储

划分为 train/val/test

In [13]:
part_num = {
    'train': 200000,
    'val': 1000,
    'test': 1000,
}

print('数据集划分：')
display(part_num)

数据集划分：


{'train': 200000, 'val': 1000, 'test': 1000}

随机种子:

In [14]:
%%time

seeds = []
seeds.extend(chain.from_iterable(
    repeat(k, v) for k, v in part_num.items()
))
seeds.extend(repeat(None, len(samples)-sum(part_num.values())))
random.shuffle(seeds)

assert len(seeds) == len(samples)

CPU times: user 177 ms, sys: 0 ns, total: 177 ms
Wall time: 176 ms


保存为 NVIDIA/Megatron-LM 的 Loose Json 格式，使用 `"text"` 作为 Key

In [15]:
%%time

output_files = {
    k: os.path.join('..', 'data', f'xinliqa_191030.{k}.json')
    for k in part_num.keys()
}

print('输出文件:')
display(output_files)

lines_iter = chain.from_iterable(map(open, corpus_files))

with ExitStack() as stack:
    files = {
        k: stack.enter_context(open(v, 'w'))
        for k, v in output_files.items()
    }
    for seed, string in tqdm(zip(seeds, samples), total=len(samples)):
        fp = files.get(seed)
        if not fp:
            continue
        print(
            json.dumps({'text': string}, ensure_ascii=False),
            file=fp
        )


输出文件:


{'train': '../data/xinliqa_191030.train.json',
 'val': '../data/xinliqa_191030.val.json',
 'test': '../data/xinliqa_191030.test.json'}


CPU times: user 3.37 s, sys: 511 ms, total: 3.88 s
Wall time: 4.34 s


In [16]:
s = ' '.join(output_files.values())

!wc -l {s}
print()

!du -h {s}
print()

   200000 ../data/xinliqa_191030.train.json
     1000 ../data/xinliqa_191030.val.json
     1000 ../data/xinliqa_191030.test.json
   202000 总用量

243M	../data/xinliqa_191030.train.json
1.3M	../data/xinliqa_191030.val.json
1.3M	../data/xinliqa_191030.test.json

